In [1]:
import json

In [2]:
shapes = []
smolrange = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 16, 17]
bigrange = smolrange + [19, 23, 31]
for x in bigrange:
    for y in bigrange:
        if y > x:
            continue
        for z in smolrange:
            if z > y:
                continue
            flat = False
            if x > 8 or y > 8 or z > 8:
                flat = True
            if flat and z > 8:
                continue
            shapes.append((x, y, z))


In [3]:
partlist = []
rend_files = {}
obj_files = {}
blockUuids = []

In [4]:
from uuid import uuid4
with open('blockUuidsOfOtherPartSizes.json', 'r') as f:
    existing_uuids = json.load(f)

In [5]:
def generate_cube(shape):
    return f"""# Blender 4.1.1
# www.blender.org
o Cube
v {0.5*shape[0]}00000 {0.5*shape[1]}00000 {-0.5*shape[2]}00000
v {0.5*shape[0]}00000 {-0.5*shape[1]}00000 {-0.5*shape[2]}00000
v {0.5*shape[0]}00000 {0.5*shape[1]}00000 {0.5*shape[2]}00000
v {0.5*shape[0]}00000 {-0.5*shape[1]}00000 {0.5*shape[2]}00000
v {-0.5*shape[0]}00000 {0.5*shape[1]}00000 {-0.5*shape[2]}00000
v {-0.5*shape[0]}00000 {-0.5*shape[1]}00000 {-0.5*shape[2]}00000
v {-0.5*shape[0]}00000 {0.5*shape[1]}00000 {0.5*shape[2]}00000
v {-0.5*shape[0]}00000 {-0.5*shape[1]}00000 {0.5*shape[2]}00000
vn -0.0000 1.0000 -0.0000
vn -0.0000 -0.0000 1.0000
vn -1.0000 -0.0000 -0.0000
vn -0.0000 -1.0000 -0.0000
vn 1.0000 -0.0000 -0.0000
vn -0.0000 -0.0000 -1.0000
vt 0.625000 0.500000
vt 0.875000 0.500000
vt 0.875000 0.750000
vt 0.625000 0.750000
vt 0.375000 0.750000
vt 0.625000 1.000000
vt 0.375000 1.000000
vt 0.375000 0.000000
vt 0.625000 0.000000
vt 0.625000 0.250000
vt 0.375000 0.250000
vt 0.125000 0.500000
vt 0.375000 0.500000
vt 0.125000 0.750000
s 0
f 1/1/1 5/2/1 7/3/1 3/4/1
f 4/5/2 3/4/2 7/6/2 8/7/2
f 8/8/3 7/9/3 5/10/3 6/11/3
f 6/12/4 2/13/4 4/5/4 8/14/4
f 2/13/5 1/1/5 3/4/5 4/5/5
f 6/11/6 5/10/6 1/1/6 2/13/6
"""

In [6]:
for shape in shapes:
    shape_string = f"{shape[0]}x{shape[1]}x{shape[2]}"
    uuid = existing_uuids.get(shape_string, str(uuid4()))
    existing_uuids[shape_string] = uuid
    part = {
        "box": {
            "x": shape[0],
            "y": shape[1],
            "z": shape[2]
        },
        "name": f"Silicon{shape_string}",
        "physicsMaterial": "Default",
        "renderable": f"$CONTENT_DATA/Renderable/Rends/block{shape_string}.rend",
        "rotationSet": "Default",
        "scripted": {
            "classname": "SiliconBlock",
            "filename": "$CONTENT_DATA/Scripts/silicon/SiliconBlock.lua"
        },
        "showInInventory": False,
        "uuid": uuid,
        "ratings": {
            "density": 2,
            "durability": 3,
            "friction": 5,
            "buoyancy": 2
        },
    }
    partlist.append(part)
    rend_file = {
        "lodList": [
            {
                "mesh": f"$CONTENT_DATA/Renderable/Models/block{shape_string}.obj",
                "subMeshList" : [
                    {
                    "material" : "DifAsgNor",
                    "textureList" : [
                        "$CONTENT_DATA/Renderable/Textures/crab_dif.png",
                        "$CONTENT_DATA/Renderable/Textures/default_asg.png",
                        "$CONTENT_DATA/Renderable/Textures/default_nor.png"
                    ]
                    }
                ]
            }
        ]
    }
    rend_files[f"block{shape_string}.rend"] = rend_file
    obj_file = generate_cube(shape)
    obj_files[f"block{shape_string}.obj"] = obj_file
    blockUuids.append(f"[\"{shape_string}\"] = \"{uuid}\"")

In [7]:
shapeset = {
    "partList": partlist,
}
with open('../Objects/Database/ShapeSets/silicon.shapeset', 'w') as f:
    f.write(json.dumps(shapeset, indent=4))
for filename, content in rend_files.items():
    with open('Rends/'+filename, 'w') as f:
        f.write(json.dumps(content, indent=4))

for filename, content in obj_files.items():
    with open('Models/'+filename, 'w') as f:
        f.write(content)

with open('blockUuids.lua', 'w') as f:
    f.write("sm.MTFastLogic.SiliconBlocksShapeDB.sizeToUuid = {\n")
    for line in blockUuids:
        f.write(f"    {line},\n")
    f.write("}\n")

with open('blockUuidsOfOtherPartSizes.json', 'w') as f:
    f.write(json.dumps(existing_uuids, indent=4))